In [17]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, accuracy_score, classification_report
import matplotlib.pyplot as plt
import numpy as np

In [18]:
# 데이터 로드
train_df = pd.read_csv('train_for_NLP.csv')
test_df = pd.read_csv('test_cleaned.csv')

# train 데이터에서 특성과 레이블 분리
X_train = train_df['combined_str']
y_train = train_df['target']

# test 데이터의 특성
X_test = test_df['combined_str']

In [19]:
# CountVectorizer 초기화
vectorizer = CountVectorizer()

# 학습 데이터를 사용하여 BoW 모델을 학습하고 임베딩
X_train_bow = vectorizer.fit_transform(X_train)

# test 데이터에 동일한 벡터라이저를 적용
X_test_bow = vectorizer.transform(X_test)

In [20]:
# 하이퍼파라미터 그리드 설정
param_grid = {
    'C': [0.001, 0.01, 0.1, 0.5, 1, 3],  # 규제 강도
    'penalty': ['l1', 'l2'],       # 규제 유형
    'solver': ['liblinear'] # 최적화 알고리즘
}

# 로지스틱 회귀 모델 초기화
model = LogisticRegression(max_iter=1000)

# GridSearchCV 초기화
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_log_loss', verbose=1)

# 모델 학습 및 최적 파라미터 찾기
grid_search.fit(X_train_bow, y_train)

# 최적 파라미터 확인
print("Best Parameters:", grid_search.best_params_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}


In [21]:
# 최적 모델로 test 데이터 예측 수행 (확률값)
best_model = grid_search.best_estimator_
y_test_pred_prob = best_model.predict_proba(X_test_bow)[:, 1]  # 확률 예측

# 확률을 이진 값으로 변환 (임계값 0.5)
y_test_pred = (y_test_pred_prob >= 0.5).astype(int)

submission_df = pd.read_csv("sample_submission.csv")
submission_df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [22]:
submission_df['target'] = y_test_pred

In [23]:
submission_df.to_csv("submit_BoW_Logistic.csv", index=False)

In [24]:
# 추가로, train 데이터에 대해 성능을 평가할 수도 있습니다.
y_train_pred_prob = best_model.predict_proba(X_train_bow)[:, 1]
train_loss = log_loss(y_train, y_train_pred_prob)
print(f"Train Log Loss: {train_loss:.4f}")

Train Log Loss: 0.5479
